In [1]:
import open3d as o3d
import numpy as np
import cv2

# Don't show the debug information
o3d.utility.set_verbosity_level(o3d.utility.VerbosityLevel.Error)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# 3D
<p style="text-align:center;">
<img src="occhiali.png" alt="drawing" width="500" style="text-align:center;"/>
</p>

## Rappresentazione immagini 2D vs immagini 3D


<p style="text-align:center;">
<img src="pixelVSvoxel.jpg" alt="drawing" width="500" style="text-align:center;"/>
</p>

In [2]:
img = cv2.imread("occhiali.png")
print("img shape: ", img.shape)

img shape:  (349, 778, 3)


## PointCloud
<p style="text-align:center;">
<img src="point_cloud.png" alt="drawing" width="400" style="text-align:center;"/>
</p>

In [3]:
import numpy as np

# Generazione di 5000 punti (x, y, z) casuali in [0, 1]
points = np.random.rand(10000, 3)
print("points shape: ", points.shape)
center_of_mass = np.mean(points, axis=0)
print("center_of_mass: ", center_of_mass)
distance_from_center = np.linalg.norm(points - center_of_mass, axis=1)

# Colorazione dei punti in base alla distanza dal centro
colors = np.zeros((10000, 3))
colors[:, :] = distance_from_center.reshape(-1, 1)


pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)
pcd.colors = o3d.utility.Vector3dVector(colors)

# Creazione del sistema di riferimento
mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.6, origin=[0, 0, 0])
o3d.visualization.draw_geometries([pcd, mesh_frame])


points shape:  (10000, 3)
center_of_mass:  [0.49877072 0.49762371 0.50654451]


# Mesh

In [4]:
bob = o3d.io.read_triangle_mesh("bob/bob_tri.obj", True)
bob.paint_uniform_color([0.8, 0.7, 0.1])

o3d.visualization.draw_geometries([bob])

In [5]:
print("Vertices:")
print(np.asarray(bob.vertices))
print("\n\nTriangles:")
print(np.asarray(bob.triangles))

Vertices:
[[-6.58735037e-02 -2.40733996e-01 -8.38036016e-02]
 [-7.08061010e-02 -2.40702003e-01 -6.75237030e-02]
 [-6.40307963e-02 -2.71822989e-01 -8.27094018e-02]
 ...
 [ 4.69967991e-01 -1.52025998e-01  1.16415003e-10]
 [-1.86077997e-01  9.75613967e-02 -1.16415003e-10]
 [-6.77003026e-01  4.03037015e-03  8.41544025e-18]]


Triangles:
[[   0    1    2]
 [   3    2    1]
 [   4    5    6]
 ...
 [5632 5625 5629]
 [5632 5634 5620]
 [5093 5620 5634]]


# Normali delle facce


In [6]:
bob = o3d.io.read_triangle_mesh("bob/bob_tri.obj", True)
bob.paint_uniform_color([0.8, 0.7, 0.1])
bob.compute_triangle_normals()

o3d.visualization.draw_geometries([bob])

# Estensione ai vertici della mesh

In [7]:
bob = o3d.io.read_triangle_mesh("bob/bob_tri.obj", True)

vertices = np.asarray(bob.vertices)

bob_pcd = o3d.geometry.PointCloud()
bob_pcd.points = o3d.utility.Vector3dVector(vertices)

o3d.visualization.draw_geometries([bob_pcd])

# Estimate vertex normals

$$
\Sigma = \frac{1}{n} \sum_{i=1}^{n} (\mathbf{x}_i - \bar{\mathbf{x}}) (\mathbf{x}_i - \bar{\mathbf{x}})^T
$$

In [8]:
print(vertices.shape)
# compute bounding box
min_bound = vertices.min(axis=0)
max_bound = vertices.max(axis=0)
print("min_bound", min_bound)
print("max_bound", max_bound)

# compute max distance between two points
diameter = np.linalg.norm(max_bound - min_bound)
print("diameter", diameter)

(5647, 3)
min_bound [-0.91293401 -0.51128101 -0.73159802]
max_bound [0.94304299 0.46125001 0.73159802]
diameter 2.555662312461616


In [13]:
bob_pcd.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=1, max_nn=20))
# change points size in visualization
o3d.visualization.draw_geometries([bob_pcd])

In [14]:
central_pcd = o3d.io.read_point_cloud("center.ply")
o3d.visualization.draw_geometries([central_pcd])

# Matrice di rototraslazione
Una generica matrice di rototraslazione è definita come segue:

<p style="text-align:center;">
<img src="traslazione.png" alt="drawing" width="600" style="text-align:center;"/>
</p>

### Componente di Traslazione
Prendendo come esempio due generiche point cloud, vediamo ora come definire una generica traslazione che permette di allineare una delle due point cloud sull'altra:

In [16]:
# SITUAZIONE INIZIALE -> Point cloud disallineate

pcl = o3d.geometry.PointCloud()
pcl.points = o3d.utility.Vector3dVector((np.random.randn(500,3)
))
pcl.paint_uniform_color((1,0,0))

reference_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=3, origin=[0, 0, 0])

o3d.visualization.draw_geometries([pcl, reference_frame])

traslazione = np.asarray([[1.0, 0.0, 0.0, 3.0],
                         [0.0, 1.0, 0.0, 0.0],
                         [0.0, 0.0, 1.0, 0.0],
                         [0.0, 0.0, 0.0, 1.0]])

transformed = o3d.geometry.PointCloud()
transformed.points = pcl.points
transformed.transform(traslazione)
transformed.paint_uniform_color((0,0,1))

o3d.visualization.draw_geometries([pcl, transformed, reference_frame])

In [17]:
# SITUAZIONE FINALE -> Dopo aver applicato la traslazione

traslazione = np.asarray([[1.0, 0.0, 0.0, -3.02],
                         [0.0, 1.0, 0.0, 0.0],
                         [0.0, 0.0, 1.0, 0.0],
                         [0.0, 0.0, 0.0, 1.0]])
transformed.transform(traslazione)
o3d.visualization.draw_geometries([pcl, transformed, reference_frame])

Oltre alla traslazione, come nel nostro esempio, dovremmo intervenire sulla matrice $R$ di rotazione.
A questo scopo occorre sapere come costruirla opportunamente.


# Matrici di rotazione

In matematica, e in particolare in geometria, una rotazione è una trasformazione del piano o dello spazio euclideo che sposta gli oggetti in modo rigido rispetto ad una retta chiamata asse di rotazione (in genere un asse del sistema di riferimento), di un angolo $\theta$  


Per evitare ambiguità, si fissa una direzione dell'asse, e si considera la rotazione dell'angolo $\theta$ effettuata in senso antiorario rispetto all'asse orientato. 

Ad esempio per definire una rotazione nello spazio 3D rispetto all'asse Y di un sistema di riferimento ortonormale, è possibile utilizzare una matrice 3x3 così definita: 

$$ R=\begin{bmatrix} \cos{\theta} & 0 & \sin{\theta} \\ 0 & 1 & 0 \\ -\sin{\theta} & 0 & \cos{\theta} \end{bmatrix}$$

Ad ogni punto verrà quindi applicata la precedente rotazione:
$$ \begin{bmatrix} x' \\ y' \\ z' \end{bmatrix}=\begin{bmatrix} \cos{\theta} & 0 & \sin{\theta} \\ 0 & 1 & 0 \\ -\sin{\theta} & 0 & \cos{\theta} \end{bmatrix}\begin{bmatrix} x \\ y \\ z \end{bmatrix}$$



In [22]:
# Artificial example 

# Create a point cloud and a copy of it with small random noise, translated and rotated
reference_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=3, origin=[0, 0, 0])
pcd_r = o3d.geometry.PointCloud()
pcd_r.points = o3d.utility.Vector3dVector(np.random.randn(1000, 3)*0.2+np.array([1,1,0]))
pcd_r.paint_uniform_color((1,0.5,0))

pcd_l = o3d.geometry.PointCloud()
pcd_l.points = o3d.utility.Vector3dVector(np.random.randn(1000, 3)*0.2+np.array([-1,1,0])+np.random.randn(3)*0.001)
pcd_l.paint_uniform_color((0,1,0))

rot_matrix = o3d.geometry.get_rotation_matrix_from_xyz((0,  np.pi/6, 0))
pcd_l.rotate(rot_matrix, center=(0,0,0))
o3d.visualization.draw_geometries([pcd_r, pcd_l, reference_frame])

In [45]:
o3d.visualization.draw_geometries([pcd_r, pcd_l, reference_frame])


rot_matrix = o3d.geometry.get_rotation_matrix_from_xyz((0,  -np.pi/6, 0))
pcd_l.rotate(rot_matrix, center=(0,0,0))
o3d.visualization.draw_geometries([pcd_r, pcd_l, reference_frame])
pcd_l.translate((2,0,0))
o3d.visualization.draw_geometries([pcd_r, pcd_l, reference_frame])

# ICP - Iterative Closest Point

<p style="text-align:center;">
<img src="icpgif.webp" alt="drawing" width="600" style="text-align:center;"/>
</p>

In [18]:
# Artificial example 

# Create a point cloud and a copy of it with small random noise, translated and rotated
reference_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=3, origin=[0, 0, 0])
x = np.random.randn(1000)
y = np.random.randn(1000)
z = np.random.randn(1000)*0.1
points_r = np.array([x, y, z]).T

pcd_r = o3d.geometry.PointCloud()
pcd_r.points = o3d.utility.Vector3dVector(points_r)
pcd_r.paint_uniform_color((1,0.5,0))

points_l = np.array([x, y, z]).T + np.random.randn(1000, 3)*0.1 + np.array([2, 0, 0])
pcd_l = o3d.geometry.PointCloud()
pcd_l.points = o3d.utility.Vector3dVector(points_l)
pcd_l.paint_uniform_color((0,1,0))

rot_matrix = o3d.geometry.get_rotation_matrix_from_xyz((0,  np.pi/6, 0))
pcd_l.rotate(rot_matrix, center=(0,0,0))
o3d.visualization.draw_geometries([pcd_r, pcd_l, reference_frame])

In [19]:
# Apply ICP to align the two point clouds

threshold = 0.5

initial_transformation = np.eye(4)
rot_matrix = o3d.geometry.get_rotation_matrix_from_xyz((0,  -np.pi/3, 0))
translation = np.array([-1, 0, 0])
initial_transformation[:3,:3] = rot_matrix
initial_transformation[:3,3] = translation

reg_p2p = o3d.pipelines.registration.registration_icp(pcd_l, pcd_r, threshold, initial_transformation, o3d.pipelines.registration.TransformationEstimationPointToPoint())
print(reg_p2p)
print("Transformation is:")
print(reg_p2p.transformation)

o3d.visualization.draw_geometries([pcd_r, pcd_l, reference_frame])

pcd_l.transform(reg_p2p.transformation)

o3d.visualization.draw_geometries([pcd_r, pcd_l, reference_frame])

RegistrationResult with fitness=9.900000e-01, inlier_rmse=1.510924e-01, and correspondence_set size of 990
Access transformation to get result.
Transformation is:
[[ 8.70177049e-01 -1.72635980e-03 -4.92736160e-01 -1.39356853e+00]
 [ 2.00462191e-03  9.99997990e-01  3.65705779e-05 -3.58268976e-02]
 [ 4.92735106e-01 -1.01957258e-03  8.70178761e-01  1.16391367e-02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [20]:
def get_random_transformation():
    x_rot = np.random.rand()*2*np.pi
    y_rot = np.random.rand()*2*np.pi
    z_rot = np.random.rand()*2*np.pi
    translation = np.random.rand(3)-1
    rot_matrix = o3d.geometry.get_rotation_matrix_from_xyz((x_rot, y_rot, z_rot))

    transformation = np.eye(4)
    transformation[:3, :3] = rot_matrix
    transformation[:3, 3] = translation

    return transformation

def flip_180_around_x():
    transformation = np.eye(4)
    transformation[1, 1] = -1
    transformation[2, 2] = -1

    return transformation

In [21]:
# apply ICP to align the two point clouds and visualize step by step

def draw_registration_result(source, target, transformation):
    source_temp = source.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target])

def step_by_step_ICP(pcd2, pcd):
    threshold = 0.5
    initial_transformation = np.eye(4)
    for i in range(15):
        reg_p2p = o3d.pipelines.registration.registration_icp(pcd2, pcd, threshold, initial_transformation, 
                                                        o3d.pipelines.registration.TransformationEstimationPointToPoint(),
                                                        o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=10))
        draw_registration_result(pcd2, pcd, reg_p2p.transformation)
        initial_transformation = reg_p2p.transformation


In [25]:
# Load a mesh and sample from it to create a point cloud
bob = o3d.io.read_triangle_mesh("bob/bob_tri.obj", True)

sampled = bob.sample_points_poisson_disk(number_of_points=10000)
sampled.paint_uniform_color([0.8, 0.7, 0.1])

o3d.visualization.draw_geometries([sampled])

In [29]:
pcd = sampled
pcd2 = o3d.geometry.PointCloud()
pcd2.points = pcd.points
pcd2.paint_uniform_color([0.1, 0.1, 0.7])
pcd2.transform(get_random_transformation())

o3d.visualization.draw_geometries([pcd, pcd2])

In [30]:
step_by_step_ICP(pcd2, pcd)

In [35]:
pcd3 = bob.sample_points_poisson_disk(number_of_points=2000)
pcd3.paint_uniform_color([0.1, 0.1, 0.7])
pcd3.transform(get_random_transformation())

o3d.visualization.draw_geometries([pcd, pcd3])

In [36]:
step_by_step_ICP(pcd3, pcd)